# Solving a reaction diffusion PDE on a GAMer mesh using FEniCS  

This tutorial will walk you through the process of setting up and solving a reaction diffusion partial differential equation (PDE) using a mesh imported from GAMer via FEniCS

In [1]:
import pygamer
import dolfin as d
import math

## Generating the mesh using GAMer

In [2]:
# generate the surface mesh for a cube
smesh = pygamer.surfacemesh.cube(5)

In [ ]:
# Mark all faces on one side of the cube
for fid in smesh.faceIDs:
    isInflux = True # flag to mark faces
    indices = fid.indices()
    for idx in indices:
        v = smesh.getVertex([idx])
        if math.isclose(v.data().position[0],-1.0):
            toMark = False
            
    if isInflux:
        fid.data().marker = 2
    #else:
    #    fid.data().marker = 3

In [ ]:
# construct the volumetric mesh
vmesh = pygamer.makeTetMesh([smesh], "q1.3/10O8/7AC")

In [ ]:
# write volumetric mesh into dolfin readable format
pygamer.writeDolfin('outputTetMesh.xml', vmesh)

## Setup the model problem in FEniCS

### Model problem: strong formulation

Find $u(\mathbf{x},t)$ such that

\begin{align}
\frac{\partial u}{\partial t} &= D\nabla^2 u + f ~~\text{in}~~ \Omega \\
D(n \cdot \nabla u) &= j_\text{in} ~~\text{on}~~ \partial\Omega_\text{in} \\
D(n \cdot \nabla u) &= j_\text{out} ~~\text{on}~~ \partial\Omega_\text{out} \\
\partial\Omega &= \partial\Omega_\text{in} \cup \partial\Omega_\text{out}
\end{align}

where $f=-0.5$ is a constant decay rate, $D=5.0$ is the diffusion coefficient, and $j_\text{in}=3.0$ and $j_\text{out}=-1.0$ are the influx/outflux rates for the Neumann boundary conditions.

In [3]:
# Model parameters
D    = 5 # diffusion coefficient
u0   = d.Constant(0.0) # initial condition
dt   = 0.1 # time step size
T    = 5.0 # final time
jin  = 3.0 # influx rate
jout = -1.0 # outflux rate
f    = -0.5 # decay rate

In [4]:
numsteps = round(T/dt)

### Model problem: weak/variational formulation

We can multiply the governing PDE by a test function, $v$, coming from a function space $V$, integrate over the domain and apply the divergence theorem to obtain the following variational formulation:

\begin{equation}
\int_\Omega \frac{\partial u}{\partial t}v \,dx = -\int_\Omega D\nabla u \cdot \nabla v \,dx + \int_{\partial\Omega} D(n\cdot\nabla u)v \,ds + \int_\Omega fv \,dx ~~\text{in}~~ \Omega 
\end{equation}

We discretize the time-derivative with a backward Euler scheme for stability purposes
$$\frac{\partial u}{\partial t} \approx \frac{u - u_n}{\Delta t}$$
where $u_n$ represents the (known) solution computed at the previous timestep. We further simplify the variational formulation by inserting the Neumann boundary conditions and using the shorthand notation where $\langle \cdot , \cdot \rangle$ represents the inner-product over $\Omega$ and $\langle \cdot , \cdot \rangle_{\partial\Omega_\text{in}}$ represents the inner-product on the boundary $\partial\Omega_\text{in}$; terms are separated by their dependence on $u$:

\begin{equation}
\langle u,v \rangle  + D\Delta t \langle \nabla u,\nabla v \rangle = \Delta t\langle f,v \rangle +  \Delta t \langle j_\text{in},v \rangle_{\partial\Omega_\text{in}} + \Delta t \langle j_\text{out},v \rangle_{\partial\Omega_\text{out}} + \langle u_n,v \rangle 
\end{equation}

In the abstract bilinear form this is written as,
\begin{align}
a(u,v) &= \langle u,v \rangle  + D\Delta t \langle \nabla u,\nabla v \rangle \\
L(v) &= \Delta t\langle f,v \rangle + \Delta t \langle j_\text{in},v \rangle_{\partial\Omega_\text{in}} + \Delta t \langle j_\text{out},v \rangle_{\partial\Omega_\text{out}} + \langle u_n,v \rangle 
\end{align}

In [23]:
# Construct linear Lagrange function space over the mesh
dolfin_mesh = d.Mesh('outputTetMesh.xml')
V = d.FunctionSpace(dolfin_mesh,'P',1)

In [24]:
# Define trial and test functions
u = d.TrialFunction(V)
v = d.TestFunction(V)
# Known solution corresponding to previous timestep. Initialize with initial condition
un = d.interpolate(u0,V)

Earlier we marked faces on the influx boundary with 2 and faces on the outflux boundaries as 3

In [25]:
# define bilinear and linear forms
a = u*v*d.dx + D*dt*d.inner(d.grad(u),d.grad(v))*d.dx
L = f*v*d.dx + dt*jin*v*d.ds(2) + dt*jout*v*d.ds(3)+ un*v*d.dx

### Solve and store solution

In [26]:
vtkfile = d.File('solution/dolfinOut.pvd')

# store the initial condition
t = 0.0 # current time
u = d.Function(V)
#u = un
vtkfile << (un,t)

In [27]:
# Main loop
for idx in range(numsteps):
    # step forward in time
    t += dt 
    
    d.solve(a==L,u,bcs=[])
    
    # save solution to file
    vtkfile << (u,t)
    
    # assign just computed solution to un
    un.assign(u)
    
    print('Time step %d complete ...' % int(idx+1))


Time step 1 complete ...
Time step 2 complete ...
Time step 3 complete ...
Time step 4 complete ...
Time step 5 complete ...
Time step 6 complete ...
Time step 7 complete ...
Time step 8 complete ...
Time step 9 complete ...
Time step 10 complete ...
Time step 11 complete ...
Time step 12 complete ...
Time step 13 complete ...
Time step 14 complete ...
Time step 15 complete ...
Time step 16 complete ...
Time step 17 complete ...
Time step 18 complete ...
Time step 19 complete ...
Time step 20 complete ...
Time step 21 complete ...
Time step 22 complete ...
Time step 23 complete ...
Time step 24 complete ...
Time step 25 complete ...
Time step 26 complete ...
Time step 27 complete ...
Time step 28 complete ...
Time step 29 complete ...
Time step 30 complete ...
Time step 31 complete ...
Time step 32 complete ...
Time step 33 complete ...
Time step 34 complete ...
Time step 35 complete ...
Time step 36 complete ...
Time step 37 complete ...
Time step 38 complete ...
Time step 39 complete